In [1]:
import pandas as pd
import numpy as np

import lib.transformers as tf
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier


In [2]:
cwruData0 = pd.read_parquet('data/cwru0.parquet')
cwruData1 = pd.read_parquet('data/cwru1.parquet')

cwruData = pd.concat([cwruData0, cwruData1])

In [3]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

In [4]:
from skorch.callbacks import PassthroughScoring

In [5]:
torch.manual_seed(0)
device = torch.device("cuda:0") if torch.cuda.is_available() else 'cpu'

In [6]:
nz = 100  # size of the latent z vector
ngf = 32  # units of generator
ndf = 32  # units of discriminator
nc = 1  # number of channels
batch_size = 32
lr = 0.0002
beta1 = 0.5  # for adam
max_epochs = 50
ngpu = 1
isize = 32  # 32 is easier than 28 to work with
workers = 2  # for dataloader

In [7]:
from lib.model import Ganomaly, GanomalyNet

In [8]:
net = GanomalyNet(
    Ganomaly,
    module__isize = isize,
    module__nz=nz,
    module__ndf=ndf,
    module__ngf=ngf,
    module__nc=nc,
    module__ngpu=ngpu,
    
    module__w_lat = 1,
    
    criterion=nn.BCELoss,

    optimizer_gen=optim.Adam,
    optimizer_gen__lr=0.0002,
    optimizer_gen__betas=(beta1, 0.999),

    optimizer_dis=optim.Adam,
    optimizer_dis__lr=0.00002,
    optimizer_dis__betas=(beta1, 0.999),

    batch_size=batch_size,
    max_epochs=max_epochs,

    train_split=False,  # not implemented
    iterator_train__shuffle=True,
    iterator_train__num_workers=workers,
    iterator_valid__num_workers=workers,

    callbacks=[
        PassthroughScoring('loss_dis', on_train=True),
        PassthroughScoring('loss_gen', on_train=True),
        PassthroughScoring('loss_gen_fra', on_train=True),
        PassthroughScoring('loss_gen_app', on_train=True),
        PassthroughScoring('loss_gen_lat', on_train=True),
    ],
)

In [9]:
normal = tf.DataSelector(columns = ['fanEndData', 'driveEndData'], condition = ['Normal Baseline']).transform(cwruData)
inner = tf.DataSelector(columns = ['fanEndData', 'driveEndData'], condition = ['Inner Race Fault']).transform(cwruData)
ball = tf.DataSelector(columns = ['fanEndData', 'driveEndData'], condition = ['Ball Fault']).transform(cwruData)
outer = tf.DataSelector(columns = ['fanEndData', 'driveEndData'], condition = ['Outer Race Fault']).transform(cwruData)


In [10]:
params = {
    'module__w_fra': [10, 12],
}

from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(net, params, refit=False, cv=3)
from sklearn.model_selection import GridSearchCV

prep = \
Pipeline(steps=[
                ('ArrayFlattener', tf.ArrayFlattener()),
                ('DatatypeFilter', tf.DataTypeFilter(data_type = np.ndarray)),
                ('ArrayChunker', tf.ArrayChunker(1024)),
                #('ArrayEqualizer', tf.ArrayEqualizer()),
                ('ArrayFlattener2', tf.ArrayFlattener()),
                ('RandomArraySampler', tf.RandomArraySampler(400, random_state = 0)),
                #('FeatureExtractor', tf.FeatureExtractor(axis=1)),
                ('ArrayReshaper', tf.ArrayReshaper((1, 32, 32))),
                ('GANomaly', net)
               ])

test = prep.fit(normal)

  epoch    loss_dis    loss_gen    loss_gen_app    loss_gen_fra    loss_gen_lat    train_loss     dur
-------  ----------  ----------  --------------  --------------  --------------  ------------  ------
      1      0.7979      1.2161          0.1152          0.7007          0.4003        2.0140  5.7316
      2      0.7633      0.8584          0.1033          0.6144          0.1407        1.6218  5.5279
      3      0.7274      0.7168          0.0911          0.5387          0.0870        1.4442  5.5332
      4      0.6898      0.6164          0.0832          0.4718          0.0614        1.3062  5.4435
      5      0.6527      0.5433          0.0778          0.4122          0.0532        1.1959  5.4211
      6      0.6138      0.4753          0.0741          0.3591          0.0421        1.0892  5.3804
      7      0.5757      0.4212          0.0711          0.3120          0.0381        0.9969  5.5128
      8      0.5338      0.3748          0.0693          0.2698          0.0356   

In [11]:
print("normal mean:", prep.predict(normal).mean())
print("inner mean:", prep.predict(inner).mean())
print("ball mean:", prep.predict(ball).mean())
print("outer mean:", prep.predict(outer).mean())

normal mean: 0.0054088323
inner mean: 0.8073717
ball mean: 0.17156105
outer mean: 1.1220227
